# Imports

In [47]:
import pandas as pd
import numpy as np
import seaborn as sns

from collections import Counter
import pickle 
import glob
import shutil
import os
import requests
import shutil
import urllib

import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.pyplot import imread, imshow, subplots, show
style.use('seaborn')
%matplotlib inline
#graphs in svg look clearer
%config InlineBackend.figure_format = 'svg' 
import warnings
warnings.simplefilter('ignore')

import pylab as pl

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim

from os import listdir
from os.path import isfile, join

from tqdm.auto import tqdm
import os
from PIL import Image
import cv2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

# Preprocessing

In [48]:
data_path = '/root/User/_DIPLOMA/data/data_tbl_cvr_oscr_kws_small.csv'

df = pd.read_csv(data_path, index_col=0)

In [3]:
df.shape

(8784, 1037)

In [4]:
df.columns[:30]

Index(['age_access_type', 'name', 'director', 'genre', 'average_rating',
       'type', 'country', 'release_type', 'release_year', 'duration', 'actor',
       'element_uid', 'ACTOR', 'COMPOSER', 'DESIGN', 'DIRECTOR', 'EDITOR',
       'OPERATOR', 'PRODUCER', 'WRITER', 'BUDGET', 'MARKETING', 'RUS', 'USA',
       'WORLD', 'element_id', 'rating', 'джек', 'расследовать', 'подруга'],
      dtype='object')

In [5]:
df.columns[-15:]

Index(['чета', 'зебра', 'сидень', 'беатрис', 'рыбка', 'has_oscar_actor',
       'has_oscar_composer', 'has_oscar_design', 'has_oscar_director',
       'has_oscar_editor', 'has_oscar_operator', 'has_oscar_producer',
       'has_oscar_writer', 'poster_url', 'poster_url_preview'],
      dtype='object')

Get rid of keywords

In [6]:
df_cols = ['age_access_type', 'name', 'director', 'genre', 'average_rating',
       'type', 'country', 'release_type', 'release_year', 'duration', 'actor',
       'element_uid', 'ACTOR', 'COMPOSER', 'DESIGN', 'DIRECTOR', 'EDITOR',
       'OPERATOR', 'PRODUCER', 'WRITER', 'BUDGET', 'MARKETING', 'RUS', 'USA',
       'WORLD', 'element_id', 'rating', 'has_oscar_actor',
       'has_oscar_composer', 'has_oscar_design', 'has_oscar_director',
       'has_oscar_editor', 'has_oscar_operator', 'has_oscar_producer',
       'has_oscar_writer', 'poster_url', 'poster_url_preview']

In [7]:
df = df[df_cols]

In [8]:
len(df['poster_url_preview'].unique())

8699

## Additional kinopoisk

In [9]:
with open('kp_add_features.pkl', 'rb') as f:
    kp_add_features = pickle.load(f)

In [10]:
kp_add_features = kp_add_features.reset_index()
kp_add_features


,index,imdb_id,film_length,has_imax,has_3_d
0,e785baa6-f175-42b4-9e16-4319ac7991d5,tt0098382,107,False,False
1,4593737e-de9c-40df-97db-fb3cf85a08ef,tt0102975,110,False,False
2,11ba66db-e941-4c3a-8da6-d8900e56f8c7,tt9469842,98,False,False
3,3f30a2ef-53b7-40e3-954f-1bdfc38a6d17,tt1985970,90,False,False
4,cdfa700f-122d-41e5-b8dc-9c6813bab6d2,tt0840305,90,False,False
...,...,...,...,...,...
10912,43bddcc0-db12-428e-9c65-f5606aa4b989,None,None,False,False
10913,ac0c9152-c070-43a0-bb99-7e72764b7c0a,None,None,False,False
10914,ce8d2906-a33f-4140-9ffe-e091990d71a4,tt4209788,140,False,False
10915,0bfb5a13-50fc-48f3-b51a-a30971024651,None,None,False,False


In [11]:
# kp_add_features = kp_add_features[['index', 'has_imax', 'has_3_d']]

In [12]:
kp_add_features.isna().sum()

index             0
imdb_id        2565
film_length    2371
has_imax          0
has_3_d           0
dtype: int64

In [13]:
kp_add_features['has_imax'].value_counts()

False    10757
True       160
Name: has_imax, dtype: int64

In [14]:
kp_add_features['has_3_d'].value_counts()

False    10594
True       323
Name: has_3_d, dtype: int64

In [15]:
kp_add_features.rename(columns={"index": "element_uid"}, inplace=True)

**Unite datasets**

In [16]:
df.shape

(8784, 37)

In [17]:
df = df.merge(kp_add_features, on='element_uid')
df.shape

(8784, 41)

### Generate feature (has a movie sequels or prequels?)

In [18]:
seq_df = pd.read_csv('/root/User/_DIPLOMA/data/Movies_with_sequels.csv', encoding= 'unicode_escape')

In [19]:
seq_df

,Position,Const,Created,Modified,Description,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,Directors
0,1,tt0145487,03-11-2011,03-11-2011,NaN,Spider-Man,https://www.imdb.com/title/tt0145487/,movie,7.3,121.0,2002.0,"Action, Adventure, Sci-Fi",670777.0,30-04-2002,Sam Raimi
1,2,tt0316654,03-11-2011,03-11-2011,NaN,Spider-Man 2,https://www.imdb.com/title/tt0316654/,movie,7.3,127.0,2004.0,"Action, Adventure, Sci-Fi",528482.0,25-06-2004,Sam Raimi
2,3,tt0413300,03-11-2011,03-11-2011,NaN,Spider-Man 3,https://www.imdb.com/title/tt0413300/,movie,6.2,139.0,2007.0,"Action, Adventure, Sci-Fi",485814.0,03-04-2007,Sam Raimi
3,4,tt0133093,03-11-2011,03-11-2011,NaN,The Matrix,https://www.imdb.com/title/tt0133093/,movie,8.7,136.0,1999.0,"Action, Sci-Fi",1615984.0,24-03-1999,"Lilly Wachowski, Lana Wachowski"
4,5,tt0234215,03-11-2011,03-11-2011,NaN,The Matrix Reloaded,https://www.imdb.com/title/tt0234215/,movie,7.2,138.0,2003.0,"Action, Sci-Fi",502256.0,07-05-2003,"Lilly Wachowski, Lana Wachowski"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,1110,tt1560220,02-02-2020,02-02-2020,NaN,Zombieland: Double Tap,https://www.imdb.com/title/tt1560220/,movie,6.7,99.0,2019.0,"Action, Comedy, Horror",113335.0,09-10-2019,Ruben Fleischer
1070,1111,tt0081505,06-03-2020,06-03-2020,NaN,The Shining,https://www.imdb.com/title/tt0081505/,movie,8.4,146.0,1980.0,"Drama, Horror",857784.0,23-05-1980,Stanley Kubrick
1071,1112,tt5606664,06-03-2020,06-03-2020,NaN,Doctor Sleep,https://www.imdb.com/title/tt5606664/,movie,7.4,152.0,2019.0,"Drama, Fantasy, Horror, Thriller",106753.0,30-10-2019,Mike Flanagan
1072,1113,tt0276919,22-05-2020,22-05-2020,NaN,Dogville,https://www.imdb.com/title/tt0276919/,movie,8.0,178.0,2003.0,"Crime, Drama",133780.0,19-05-2003,Lars von Trier


In [20]:
seq_df = seq_df.rename(columns={'Const': 'imdb_id'})

seq_df

,Position,imdb_id,Created,Modified,Description,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,Directors
0,1,tt0145487,03-11-2011,03-11-2011,NaN,Spider-Man,https://www.imdb.com/title/tt0145487/,movie,7.3,121.0,2002.0,"Action, Adventure, Sci-Fi",670777.0,30-04-2002,Sam Raimi
1,2,tt0316654,03-11-2011,03-11-2011,NaN,Spider-Man 2,https://www.imdb.com/title/tt0316654/,movie,7.3,127.0,2004.0,"Action, Adventure, Sci-Fi",528482.0,25-06-2004,Sam Raimi
2,3,tt0413300,03-11-2011,03-11-2011,NaN,Spider-Man 3,https://www.imdb.com/title/tt0413300/,movie,6.2,139.0,2007.0,"Action, Adventure, Sci-Fi",485814.0,03-04-2007,Sam Raimi
3,4,tt0133093,03-11-2011,03-11-2011,NaN,The Matrix,https://www.imdb.com/title/tt0133093/,movie,8.7,136.0,1999.0,"Action, Sci-Fi",1615984.0,24-03-1999,"Lilly Wachowski, Lana Wachowski"
4,5,tt0234215,03-11-2011,03-11-2011,NaN,The Matrix Reloaded,https://www.imdb.com/title/tt0234215/,movie,7.2,138.0,2003.0,"Action, Sci-Fi",502256.0,07-05-2003,"Lilly Wachowski, Lana Wachowski"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,1110,tt1560220,02-02-2020,02-02-2020,NaN,Zombieland: Double Tap,https://www.imdb.com/title/tt1560220/,movie,6.7,99.0,2019.0,"Action, Comedy, Horror",113335.0,09-10-2019,Ruben Fleischer
1070,1111,tt0081505,06-03-2020,06-03-2020,NaN,The Shining,https://www.imdb.com/title/tt0081505/,movie,8.4,146.0,1980.0,"Drama, Horror",857784.0,23-05-1980,Stanley Kubrick
1071,1112,tt5606664,06-03-2020,06-03-2020,NaN,Doctor Sleep,https://www.imdb.com/title/tt5606664/,movie,7.4,152.0,2019.0,"Drama, Fantasy, Horror, Thriller",106753.0,30-10-2019,Mike Flanagan
1072,1113,tt0276919,22-05-2020,22-05-2020,NaN,Dogville,https://www.imdb.com/title/tt0276919/,movie,8.0,178.0,2003.0,"Crime, Drama",133780.0,19-05-2003,Lars von Trier


In [21]:
df.shape

(8784, 41)

In [38]:
df['has_sequel'] = df['imdb_id'].apply(lambda x: 1 if x in seq_df['imdb_id'].values else 0)

In [39]:
df['has_sequel'].value_counts()

0    8245
1     539
Name: has_sequel, dtype: int64

In [43]:
df[(df['has_sequel'] == 1) & (df['release_year'] > 2000)]['release_year'].unique()

array([2009, 2016, 2002, 2013, 2012, 2017, 2008, 2001, 2007, 2004, 2003,
       2006, 2011, 2005, 2015, 2019, 2014, 2010, 2018, 2020])

In [44]:
df.columns

Index(['age_access_type', 'name', 'director', 'genre', 'average_rating',
       'type', 'country', 'release_type', 'release_year', 'duration', 'actor',
       'element_uid', 'ACTOR', 'COMPOSER', 'DESIGN', 'DIRECTOR', 'EDITOR',
       'OPERATOR', 'PRODUCER', 'WRITER', 'BUDGET', 'MARKETING', 'RUS', 'USA',
       'WORLD', 'element_id', 'rating', 'has_oscar_actor',
       'has_oscar_composer', 'has_oscar_design', 'has_oscar_director',
       'has_oscar_editor', 'has_oscar_operator', 'has_oscar_producer',
       'has_oscar_writer', 'poster_url', 'poster_url_preview', 'imdb_id',
       'film_length', 'has_imax', 'has_3_d', 'has_sequel'],
      dtype='object')

In [45]:
df.drop(['imdb_id', 'film_length'], axis=1, inplace=True)

In [46]:
with open('new_eng_data.pkl', 'wb') as f:
    pickle.dump(df, f)

In [22]:
ddd = df.merge(seq_df, on='imdb_id', how='left')

ddd.shape

(8784, 55)

In [23]:
ddd

,age_access_type,name,director,genre,average_rating,type,country,release_type,release_year,duration,actor,element_uid,ACTOR,COMPOSER,DESIGN,DIRECTOR,EDITOR,OPERATOR,PRODUCER,WRITER,BUDGET,MARKETING,RUS,USA,WORLD,element_id,rating,has_oscar_actor,has_oscar_composer,has_oscar_design,has_oscar_director,has_oscar_editor,has_oscar_operator,has_oscar_producer,has_oscar_writer,poster_url,poster_url_preview,imdb_id,film_length,has_imax,has_3_d,Position,Created,Modified,Description,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,Directors
0,12,Звёздный путь 5: Последний рубеж,['cff3362f-ad91-498d-80b3-ea3ab8ec65cf'],"['Sci-Fi', 'Action', 'Thriller', 'Adventure']",6.21,MOVIE,['usa'],RELEASE,1989,6420000.0,"['cff3362f-ad91-498d-80b3-ea3ab8ec65cf', '21c7a451-d42c-4c4b-90ee-7bc9658f9361', '1db4ce9a-3623-434a-8603-3f144651b6fd', '8cf3b1bc-37c0-4fab-875a-20974fb08178', '6744ca09-1710-4613-8661-1eb987347505']",e785baa6-f175-42b4-9e16-4319ac7991d5,"['Уильям Шетнер', 'Леонард Нимой', 'ДеФорест Келли', 'Джеймс Дуэн', 'Уолтер Кениг', 'Нишелль Николс', 'Джордж Такей', 'Дэвид Уорнер', 'Лоуренс Лакинбилл', 'Чарльз Купер', 'Синтия Гау', 'Тодд Брайант', 'Спайс Уильямс', 'Рекс Холман', 'Джордж Мёрдок', 'Джонатан Симпсон', 'Беверли Харт', 'Стив Сусскинд', 'Харви Беннетт', 'Синтия Блэйз', 'Билл Куинн', 'Мелани Шетнер', 'Майкл Берриман', 'Джин Кросс', 'Линда Фетерс', 'Конрад Хёртт', 'Мэл Меде', 'Патрик Майкл', 'Кэри Скотт', 'Майк Смитсон', 'Адриан Тафоя', 'Илона Уилсон']","['Джерри Голдсмит', 'Хиросима']","['Херман Ф. Зиммерман', 'Нило Родис-Джамеро', 'Джон М. Двайер']",['Уильям Шетнер'],['Питер Е. Бергер'],['Эндрю Ласло'],"['Харви Беннетт', 'Брук Бретон', 'Мэл Эфрос', 'Джин Родденберри', 'Ральф Уинтер']","['Джин Родденберри', 'Уильям Шетнер', 'Харви Беннетт', 'Дэвид Локери']",27800000.0,NaN,NaN,52210049.0,52210049.0,e785baa6-f175-42b4-9e16-4319ac7991d5,8.339286,0,0,0,0,0,0,0,0,https://kinopoiskapiunofficial.tech/images/posters/kp/6580.jpg,https://kinopoiskapiunofficial.tech/images/posters/kp_small/6580.jpg,tt0098382,107,False,False,747.0,13-04-2013,13-04-2013,NaN,Star Trek V: The Final Frontier,https://www.imdb.com/title/tt0098382/,movie,5.5,107.0,1989.0,"Action, Adventure, Fantasy, Sci-Fi, Thriller",54509.0,09-06-1989,William Shatner
1,16,Звёздный путь 6: Неоткрытая страна,['bab6b7f4-4506-458f-9091-d567505674f2'],"['Sci-Fi', 'Action', 'Thriller', 'Adventure']",7.20,MOVIE,['usa'],RELEASE,1991,6780000.0,"['cff3362f-ad91-498d-80b3-ea3ab8ec65cf', '21c7a451-d42c-4c4b-90ee-7bc9658f9361', '1db4ce9a-3623-434a-8603-3f144651b6fd', '8cf3b1bc-37c0-4fab-875a-20974fb08178', '6744ca09-1710-4613-8661-1eb987347505']",4593737e-de9c-40df-97db-fb3cf85a08ef,"['Уильям Шетнер', 'Леонард Нимой', 'ДеФорест Келли', 'Джеймс Дуэн', 'Уолтер Кениг', 'Нишелль Николс', 'Джордж Такей', 'Ким Кэтролл', 'Дэвид Уорнер', 'Кристофер Пламмер', 'Марк Ленард', 'Грэйс Ли Уитни', 'Брок Питерс', 'Леон Рассом', 'Кертвуд Смит', 'Розанна ДеСото', 'Джон Шак', 'Майкл Дорн', 'Пол Россилли', 'Роберт Истон', 'Уильям Морган Шеппард', 'Бретт Портер', 'Джереми Робертс', 'Майкл Бофшевер', 'Анджело Тиффе', 'Борис Ли Крутоног', 'Кристиан Слэйтер', 'Иман', 'Том Морга', 'Тодд Брайант', 'Джон Блум', 'Джим Боэк', 'Карлос Цестеро', 'Эдвард Клементс', 'Кэти Джонстон', 'Маттиас Хьюз', 'Дэррил Энрикес', 'Дэвид Оранж', 'Джуди Левитт', 'Шакти', 'Майкл Снайдер', 'Джойс Агу', 'Рене Обержонуа', 'Лена Бэнкс', 'Фэйт Бертон', 'Джон Бек', 'Берри Кристиан', 'Роберт Коэн', 'Б.Дж. Дэвис', 'Дуглас Даннинг', 'Роберт Гейтвуд', 'Клэй Ходжес', 'Джеймс Мейп', 'Алан Маркус', 'Патрик Майкл', 'Деннис Отт', 'Дуглас Прайс', 'Денис Линн Робертс', 'Брайан Нэш', 'Ричард Зарштедт', 'Эрик А. Стиллвелл', 'Дж.Д. Уолтерс']",['Клифф Эйдельман'],"['Херман Ф. Зиммерман', 'Нило Родис-Джамеро', 'Доди Шепард', 'Мики С. Майклс']",['Николас Мейер'],"['Уильям Хой', 'Рональд Руз']",['Хиро Нарита'],"['Стивен-Чарльз Джаффе', 'Ральф Уинтер', 'Брук Бретон', 'Марти Хорнштейн', 'Леонард Нимой']","['Джин Родденберри', 'Леонард Нимо

In [24]:
ddd.isna().sum()

age_access_type          0
name                     0
director                 3
genre                    0
average_rating         337
type                     0
country                  0
release_type             0
release_year             0
duration                 0
actor                    0
element_uid              0
ACTOR                  110
COMPOSER               968
DESIGN                 959
DIRECTOR                98
EDITOR                1119
OPERATOR              1101
PRODUCER              1045
WRITER                 234
BUDGET                5683
MARKETING             8538
RUS                   5734
USA                   5188
WORLD                 3902
element_id               0
rating                   0
has_oscar_actor          0
has_oscar_composer       0
has_oscar_design         0
has_oscar_director       0
has_oscar_editor         0
has_oscar_operator       0
has_oscar_producer       0
has_oscar_writer         0
poster_url               0
poster_url_preview       0
i

### +500 штук!!!

In [28]:
train = pd.read_csv('/root/User/_DIPLOMA/data/train.csv')
train

id  \
0        1   
1        2   
2        3   
3        4   
4        5   
...    ...   
2995  2996   
2996  2997   
2997  2998   
2998  2999   
2999  3000   

                                                                                                                                                    belongs_to_collection  \
0     [{'id': 313576, 'name': 'Hot Tub Time Machine Collection', 'poster_path': '/iEhb00TGPucF0b4joM1ieyY026U.jpg', 'backdrop_path': '/noeTVcgpBiD48fDjFVic1Vz7ope.jpg'}]   
1     [{'id': 107674, 'name': 'The Princess Diaries Collection', 'poster_path': '/wt5AMbxPTS4Kfjx7Fgm149qPfZl.jpg', 'backdrop_path': '/zSEtYD77pKRJlUPx34BJgUG9v1c.jpg'}]   
2                                                                                                                                                                     NaN   
3                                                                                                                                                                     NaN   
4                                                                                                                                                                     NaN   
...                                                                                                                                                                   ...   
2995                                                                                                                                                                  NaN   
2996                                                                                                                                                                  NaN   
2997                                                                                                                                                                  NaN   
2998                                                                                                                                                                  NaN   
2999                                                                                                                                                                  NaN   

        budget  \
0     14000000   
1     40000000   
2      3300000   
3      1200000   
4            0   
...        ...   
2995         0   
2996         0   
2997  65000000   
2998  42000000   
2999  35000000   

                                                                                                                              genres  \
0                                                                                                     [{'id': 35, 'name': 'Comedy'}]   
1     [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 10749, 'name': 'Romance'}]   
2                                                                                                      [{'id': 18, 'name': 'Drama'}]   
3                                                                      [{'id': 53, 'name': 'Thriller'}, {'id': 18, 'name': 'Drama'}]   
4                                                                     [{'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}]   
...                                                                                                                              ...   
2995                                                                [{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]   
2996                                                                   [{'id': 18, 'name': 'Drama'}, {'id': 10402, 'name': 'Music'}]   
2997    [{'id': 80, 'name': 'Crime'}, {'id': 28, 'name': 'Action'}, {'id': 9648, 'name': 'Mystery'}, {'id': 53, 'name': 'Thriller'}]   
2998                                                                [{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]   
2999                                 [{'id': 53, 

In [33]:
train.shape, df.shape

((3000, 23), (8784, 41))

In [34]:
new_ddd = df.merge(train, on='imdb_id')

new_ddd.shape

(918, 63)

In [31]:
train.head(2).T

,0,1
id,1,2
belongs_to_collection,"[{'id': 313576, 'name': 'Hot Tub Time Machine Collection', 'poster_path': '/iEhb00TGPucF0b4joM1ieyY026U.jpg', 'backdrop_path': '/noeTVcgpBiD48fDjFVic1Vz7ope.jpg'}]","[{'id': 107674, 'name': 'The Princess Diaries Collection', 'poster_path': '/wt5AMbxPTS4Kfjx7Fgm149qPfZl.jpg', 'backdrop_path': '/zSEtYD77pKRJlUPx34BJgUG9v1c.jpg'}]"
budget,14000000,40000000
genres,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 10749, 'name': 'Romance'}]"
homepage,NaN,NaN
imdb_id,tt2637294,tt0368933
original_language,en,en
original_title,Hot Tub Time Machine 2,The Princess Diaries 2: Royal Engagement
overview,"When Lou, who has become the ""father of the Internet,"" is shot by an unknown assailant, Jacob and Nick fire up the time machine again to save their friend.","Mia Thermopolis is now a college graduate and on her way to Genovia to take up her duties as princess. Her best friend Lilly also joins her for the summer. Mia continues her 'princess lessons'- riding horses side-saddle, archery, and other royal. But her complicated life is turned upside down once again when she not only learns that she is to take the crown as queen earlier than expected..."
popularity,6.575393,8.248895


In [29]:
'sequel_name' in train.columns

False

In [26]:
import re

# list all movies that contain numbers
films_with_ints = sorted([title for title in df['name'] if re.search(r'\d+', title)])
films_with_ints

['1+1: Голливудская история',
 '10 лет спустя',
 '10 минут спустя',
 '10 причин моей ненависти',
 '10 свиданий',
 '10-летний план',
 '100 вещей и ничего лишнего',
 '100 дней на жизнь',
 '100 мужчин и одна девушка',
 '100-футовая волна',
 '101 далматинец',
 '101 далматинец',
 '101 далматинец',
 '101 далматинец 2: Приключения Патча в Лондоне',
 '102 далматинца',
 '11-11-11',
 '11.22.63',
 '12 лет рабства',
 '12 мелодий любви',
 '12 обезьян',
 '12 обезьян',
 '12 разгневанных мужчин',
 '12 раундов',
 '12 рождественских свиданий',
 '12 стульев',
 '127 часов',
 '128 ударов сердца в минуту',
 '13 грехов',
 '13 убийц',
 '13 часов: Тайные солдаты Бенгази',
 '13-й воин',
 '13-й район',
 '13-й район: Кирпичные особняки',
 '1408',
 '15 убийств',
 '16 кварталов',
 '1917',
 '1984',
 '2 сердца',
 '2+1',
 '20000 лье под водой',
 '2012',
 '2012: Цунами',
 '2025: Прелюдия к Инфуско',
 '2040: Будущее ждёт',
 '2050',
 '2067: Петля времени',
 '21 грамм',
 '21 мост',
 '22 мили',
 '2307: Операция Андроид',
 

In [ ]:
def has_prequels(data):
    data_arr = data.split(' ')
    if len(data_arr) == 2:
        if any(str.isdigit(c) for c in data):
            data = data_arr[0]
    elif len(data_arr) > 2:
        data = data_arr[0] + ' ' + data_arr[1]
        
    return len(df[df.name.str.contains(data)])

In [126]:
def get_num_prequels(data):
    data_arr = data.split(' ')
    if len(data_arr) == 2:
        if (any(str.isdigit(c) for c in data)) and (not str.isdigit(data_arr[0])):
            data = data_arr[0]
    elif len(data_arr) > 2:
        data = data_arr[0] + ' ' + data_arr[1]
        
    return len(df[df.name.str.contains(data + ' ')])

In [127]:
df['seq_amount'] = df['name'].apply(get_num_prequels)

In [136]:
df[df['seq_amount'] == 10]

,age_access_type,name,director,genre,average_rating,type,country,release_type,release_year,duration,actor,element_uid,ACTOR,COMPOSER,DESIGN,DIRECTOR,EDITOR,OPERATOR,PRODUCER,WRITER,BUDGET,MARKETING,RUS,USA,WORLD,element_id,rating,has_oscar_actor,has_oscar_composer,has_oscar_design,has_oscar_director,has_oscar_editor,has_oscar_operator,has_oscar_producer,has_oscar_writer,poster_url,poster_url_preview,has_imax,has_3_d,seq_amount
1639,18,Дикая,['75fd2918-6d72-4428-9f6b-dae185f3e8ba'],"['Drama', '0ef0aa70-f86c-4141-8054-8b39af97867d']",7.16,MOVIE,['usa'],RELEASE,2014,6900000.0,"['4fbf4a23-ff61-4e6d-8f29-0ca4a445a5d6', '38a11544-f866-4fc6-8dec-605a5746181c', 'eadc34f2-3e88-4c17-96ab-0a3fdf04cfd6', '926741fe-136f-4be8-8e0c-14f14a571506', '6513f5bf-f7e6-42dc-914e-4c71f100a30b']",808ce402-4314-483c-94e3-7ce1fdf211f5,"['Риз Уизерспун', 'Лора Дерн', 'Томас Садоски', 'Кин МакРей', 'Михил Хёйсман', 'У. Эрл Браун', 'Габи Хоффманн', 'Кевин О. Ранкин', 'Брайан Ван Холт', 'Клифф Де Янг', 'Мо', 'Уилл Кадди', 'Ли Паркер', 'Ник Эверсман', 'Аллес Мист', 'Рэнди Шульман', 'Катрин Де Прюм', 'Курт Конройд', 'Тед ДеШатле', 'Джеффри Ньюман', 'Лоррэйн Бар', 'Джерри Карлтон', 'Кевин Майкл Мур', 'Дебра Пралле', 'Грэй Юбэнк', 'Энн Сорсе', 'Чарльз Бэйкер', 'Дж.Д. Эвермор', 'Бет Холл', 'Жан Хоаг', 'Карли МакМанус', 'Арт Алексакис', 'Энн Ги Бёрд', 'Ивэн О’Тул', 'Джанин Джексон', 'Джейсон Ньюэлл', 'Бэрри О’Нил', 'Рич Моррис', 'Бобби Стрейд Линдстром', 'Роберт Алан Барнетт', 'Уилльям Нельсон', 'Тони Дуп', 'Дэн Консидайн', 'Эндрю Сэундерсон', 'Грег Джеймс', 'Орианна Геррманн', 'Брайан Борчердт', 'Эрик Д. Джонсон', 'Мэтт Паскуа', 'Эдрик Картер', 'Джон Дикерсон мл.', 'Денис МакСуини', 'Алина Гатти', 'Генри Шотуэлл', 'Шерил Стрэйд', 'Закри Эйрс', 'Джозеф Томас Бэйли', 'Сара Бэйли', 'Бретт Баррон', 'Винсент М. Бишьон', 'Ричард Дж. Борон', 'Коуди Бёрнс', 'Майлз Карден', 'Уилльям Доберт', 'Мэттью Джоэль Флуд', 'Элизабет Форнье', 'Бека Грэйс', 'Дилан Холл', 'Симос Каливас', 'Ник Келлингтон', 'Брент Кублик', 'Дилан Уэйн Лоуренс', 'Джейкоб Лупер', 'Джэми МакКолл', 'Эдвин Модлин', 'Блейн Палмер', 'Элизабет Пеннингтон', 'Тед Руни', 'Трип Росс', 'Эрик Зальстром', 'Питер Симпсон', 'Уэйн Суэт', 'Роберт Зорн']",NaN,"['Джон Пэйно', 'Хавьера Варас', 'Мелисса Брюнинг', 'Роберт Ковелман']",['Жан-Марк Валле'],"['Мартин Пенса', 'Жан-Марк Валле']",['Ив Беланже'],"['Дэвид Гринбаум', 'Бруна Папандреа', 'Билл Полад', 'Риз Уизерспун', 'Джеффри Харлэкер', 'Ник Хорнби', 'Т.К. Ноулз', 'Джон О’Грэйди', 'Нэйтан Росс', 'Чак Райант', 'Шерил Стрэйд', 'Берген Суонсон']","['Ник Хорнби', 'Шерил Стрэйд']",15000000.0,NaN,300457.0,37880356.0,52501541.0,808ce402-4314-483c-94e3-7ce1fdf211f5,8.626013,0,0,0,0,0,0,0,0,https://kinopoiskapiunofficial.tech/images/posters/kp/673428.jpg,https://kinopoiskapiunofficial.tech/images/posters/kp_small/673428.jpg,False,False,10
2850,18,Легенда о Зелёном рыцаре,['9efc3748-71bb-4904-9ff2-d61cff685bc3'],"['Drama', 'Adventure', '9fa28b61-a257-4a3e-945b-a9ef76a146d6']",6.54,MOVIE,"['usa', 'great-britain', 'canada', 'ireland']",RELEASE,2021,7440000.0,"['89cba4ef-4d2e-4489-8d0a-58a8fbd9dca1', '8527792f-8039-4173-8ad9-450e0cdf5e6c', 'ec45f9e0-041d-4a29-940f-e5fdcacee040', 'b84e590d-1afb-47fd-960e-7594d1ee05ff', 'a8cd20bd-2cb2-4fd5-b597-236eb736e873', 'fc779c5c-c6ac-4544-aca0-e6c62d74d9ca', 'e1c246c0-8e39-4cfd-93b4-618886d0cff5', 'dba749e4-7530-4560-9f7e-41f07d7ac674', '0226c9fe-5265-4d93-8013-e3ff4b525f68', 'cf7032bd-8302-48c9-b64c-3ba652e0bbab']",fa6db5cb-14cb-4278-a2d1-14038650e39e,"['Дев Патель', 'Алисия Викандер', 'Джоэл Эдгертон', 'Сарита Чоудри', 'Шон Харрис', 'Кейт Дики', 'Ралф Айнесон', 'Барри Кеоган', 'Хелена Браун', 'Эрин Келлиман', 'Анаис Риццо', 'Джо Андерсон', 'Джо Андерсон', 'Ноэль Браун', 'Нита Мишра', 'Тара МакДонах', 'Ноэль Браун', 'Афина Фриззелл', 'Марго Райт', 'Крис Макхаллем', 'Эммет О’Брайэн', 'Эмили Хетленд', 'Брендан Конрой', 'Доннча Краули', 'Энтони Моррис', 'Хелена Браун', 'Меган Тирнан', 'Рут Патель', 'Роуз Патель', 'Патрик Даффи', 'Аарон Эдо', 'Джа

In [131]:
df['seq_amount'].value_counts().sort_index()

0     4451
1     3012
2      573
3      271
4      149
5       82
6       50
7       47
8       20
9       49
10      11
11      13
12      15
13       6
14      15
16       1
20      10
22       3
24       3
29       2
44       1
Name: seq_amount, dtype: int64

In [83]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


print(cosine_sim('Звёздный 2: ответный удар', 'Звёздный '))
print(cosine_sim('Звёздный путь 6: Неоткрытая страна', 'Звёздный путь 5: Последний рубеж'))

0.37997836159100784
0.2523342014336961


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [104]:
fuzzy_matches

[('Перевозчик 2', 100),
 ('Шанхайский перевозчик', 100),
 ('Перевозчик', 100),
 ('Перевозчик: Наследие', 100),
 ('Переводчики', 86)]

In [107]:
df[df['name'].str.contains('Перевозчик 2')]['name'].values

array(['Перевозчик 2'], dtype=object)